In [1]:
import requests

# Broadband.gov APIs
Broadband.gov provides APIs to access internet availability and internet speeds by county in the US. To construct this, I use data from the availability API and get State IDs and County IDs.

These county IDs are then used to get median download and upload speeds by county. The API also provides details about lower and upper quartile upload and download speeds.

In [2]:
"""Store information about State IDs and County IDs. This will be the key for subsequent data points."""

# This collects information about broadband availability in every county in the US
url = 'https://www.broadbandmap.gov/broadbandmap/county-availability/jun2014/nation?format=json'
response = requests.get(url, timeout=100)

In [3]:
"""Create a dataframe to store these results, temporarily as a Python list"""
from collections import namedtuple
DataContainer = namedtuple('DataContainer', 
                           ['stateName', 'countyName', 'stateId', 'countyId', 'friendlyKey', 'gt50Available'])

internet_avail_data = []

for entry in response.json()['Results']:
    internet_avail_data.append(DataContainer(stateName=entry['stateName'].lower(), countyName=entry['countyName'].lower(), 
                                             stateId=entry['stateId'], countyId=entry['countyId'], 
                                             gt50Available=entry['availabilityGt50PercentFlag'], 
                                             friendlyKey='%s_%s_county' % (entry['stateName'].lower().replace(' ', '_'), 
                                                                           entry['countyName'].lower().replace(' ', '_'))))

In [4]:
"""Create a pandas dataframe to store as CSV and perform analysis"""
import pandas as pd
df = pd.DataFrame(internet_avail_data, columns=DataContainer._fields)

In [5]:
"""View first few entries in dataframe"""
print(df.head())

        stateName countyName stateId countyId  \
0  south carolina  abbeville      45    45001   
1       louisiana     acadia      22    22001   
2        virginia   accomack      51    51001   
3           idaho        ada      16    16001   
4        oklahoma      adair      40    40001   

                       friendlyKey  gt50Available  
0  south_carolina_abbeville_county           True  
1          louisiana_acadia_county           True  
2         virginia_accomack_county           True  
3                 idaho_ada_county           True  
4            oklahoma_adair_county          False  


In [6]:
"""Save dataframe to disk for later use"""
file_path = '../Data/StateCountyInternetAvailability.csv'
df.to_csv(file_path, sep=',', encoding='utf-8')

In [38]:
friendlyKeyToFips = {}
FipsToFriendlyKey = {}
for _ in df.iterrows():
    _id, row = _
    friendlyKeyToFips[row.friendlyKey] = row.countyId
    FipsToFriendlyKey[row.countyId] = row.friendlyKey

## Getting Interned speed data of each county

For each county, we can get the average Internet speeds using the speed test API in BroadbandMap.gov

In [7]:
"""Get Internet speed data of each county"""
url = 'https://www.broadbandmap.gov/broadbandmap/speedtest/county/ids/%s?format=json'

InternetSpeedContainer = namedtuple('InternetSpeedContainer', ['friendlyKey', 'countyId', 'stateId', 
                                                               'stateName', 'countyName', 'category', 
                                                               'numTests', 'medianDownload', 'medianUpload', 
                                                               'lowerQuartileDownload', 'lowerQuartileUpload', 
                                                               'upperQuartileDownload', 'upperQuartileUpload'])
internet_speed_data = []

for _ in df.iterrows():
    _id, row = _
    response = requests.get(url % row.countyId)
    if 'Results' not in response.json():
        result = {
            'accessingFrom': None,
            'numberOfTests': 0,
            'medianDownload': 0,
            'medianUpload': 0,
            'lowerQuartileDownload': 0,
            'lowerQuartileUpload': 0,
            'upperQuartileDownload': 0,
            'upperQuartileUpload': 0
        }
    else:
        results = response.json()['Results']
    for result in results:
        speed_data = InternetSpeedContainer(row.friendlyKey, row.countyId, row.stateId, row.stateName, row.countyName, 
                                            result['accessingFrom'], result['numberOfTests'], result['medianDownload'], 
                                            result['medianUpload'], result['lowerQuartileDownload'], 
                                            result['lowerQuartileUpload'], result['upperQuartileDownload'], 
                                            result['upperQuartileUpload'])
        internet_speed_data.append(speed_data)

internet_speed_df = pd.DataFrame(internet_speed_data, columns=InternetSpeedContainer._fields)

print("Top few rows of Internet Speed Data")
print(internet_speed_df.head())

internet_speed_df.to_csv('../Data/CountyInternetSpeed.csv', sep=',', encoding='utf-8')

Top few rows of Internet Speed Data
                       friendlyKey countyId stateId       stateName  \
0  south_carolina_abbeville_county    45001      45  south carolina   
1  south_carolina_abbeville_county    45001      45  south carolina   
2  south_carolina_abbeville_county    45001      45  south carolina   
3          louisiana_acadia_county    22001      22       louisiana   
4          louisiana_acadia_county    22001      22       louisiana   

  countyName           category  numTests  medianDownload  medianUpload  \
0  abbeville               Home         8         1.46094       0.68360   
1  abbeville             Mobile        29         1.37500       0.74805   
2  abbeville     Small_Business         3         5.55128       4.83496   
3     acadia  CC_library_school         1         1.43272       1.37600   
4     acadia               Home        30         3.07021       0.40723   

   lowerQuartileDownload  lowerQuartileUpload  upperQuartileDownload  \
0             

## County Internet Provider Statistics

Find the number of wireline providers and the percentage of population that hass access to the Internet

In [39]:
"""Get detailed dataset of each county - number of wireline providers and the percentage 
of the population with access to these wireline providers"""
url = 'https://www.broadbandmap.gov/broadbandmap/analyze/jun2014/summary/population/county/ids/%s?format=json'

county_provider_statistics = []

for _ in df.iterrows():
    _id, row = _
    response = requests.get(url % row.countyId)
    if 'Results' not in response.json():
        result = {}
    else:
        results = response.json()['Results'][0]
        results['countyId'] = row.countyId
        results['county'] = row.friendlyKey
        county_provider_statistics.append(results)

In [41]:
county_providers_df = pd.DataFrame.from_records(county_provider_statistics)
county_providers_df.set_index('county', inplace=True)
print("Top few rows of County Providers Statistics")
print(county_providers_df.head())
county_providers_df.to_csv('../Data/CountyProviderStatistics.csv', sep=',', encoding='utf-8')

Top few rows of County Providers Statistics
                                 anyTechnology  anyWireline  asymmetricDsl  \
county                                                                       
south_carolina_abbeville_county         1.0000       0.9942         0.4928   
louisiana_acadia_county                 1.0000       0.8419         0.7543   
virginia_accomack_county                0.9926       0.8238         0.8206   
idaho_ada_county                        1.0000       0.9843         0.9721   
oklahoma_adair_county                   0.9633       0.2474         0.2474   

                                 cableDocsis  cableOther  cableTechnology  \
county                                                                      
south_carolina_abbeville_county       0.6392      0.0000           0.6392   
louisiana_acadia_county               0.7049      0.0000           0.7049   
virginia_accomack_county              0.0000      0.0929           0.0929   
idaho_ada_county        

## Preprocessing the Voter Data
We have voter data organized by counties for the years 2008, 2012, and 2016. For the preliminary model, we can build a dataframe with all of data present in voter data and try to predict the Internet speed.

In [30]:
us_voter_county_df = pd.DataFrame.from_csv('../Data/US_County_Level_Presidential_Results_08-16.csv', sep=',')
print(us_voter_county_df.head())

                       county  total_2008  dem_2008  gop_2008  oth_2008  \
fips_code                                                                 
26041            Delta County       19064      9974      8763       327   
48295         Lipscomb County        1256       155      1093         8   
1127            Walker County       28652      7420     20722       510   
48389           Reeves County        3077      1606      1445        26   
56017      Hot Springs County        2546       619      1834        93   

           total_2012  dem_2012  gop_2012  oth_2012  total_2016  dem_2016  \
fips_code                                                                   
26041           18043      8330      9533       180       18467      6431   
48295            1168       119      1044         5        1322       135   
1127            28497      6551     21633       313       29243      4486   
48389            2867      1649      1185        33        3184      1659   
56017       

In [31]:
"""Preprocessing voter data and storing in dictionary by FIPS code as key"""
CountyVote = namedtuple('CountyVote', ['countyId', 'total_2016', 'dem_2016', 'gop_2016', 'oth_2016', 'dem_percent'])

voter_data = {}
for _ in us_voter_county_df.iterrows():
    _id, row = _
    voter_data[_id] = CountyVote(_id, row.total_2016, row.dem_2016, row.gop_2016, row.oth_2016, 
                                           row.dem_2016 / float(row.total_2016))

In [32]:
"""Preprocess FCC Internet Demographics data"""
internet_demographics_df = pd.read_csv('../Data/FCC-2016-Population-Internet.csv')
print(internet_demographics_df.head())

           county pop_without_access percent_total_pop pop_density  \
0         Alabama            985,263               20%      96.963   
1  Autauga County             13,199               22%       99.21   
2  Baldwin County             55,717               27%      128.37   
3  Barbour County             13,236               51%      29.563   
4     Bibb County             22,772               98%      37.203   

  per_capita_income urban_pop_without_access percent_urban_pop  \
0                 .                  169,154                6%   
1          $24,644                       360                1%   
2          $26,851                    22,041               19%   
3          $17,350                       608                7%   
4          $18,110                     7,323               98%   

  urban_pop_density rural_pop_without_access percent_rural_pop  \
0          1,333.15                  816,109               41%   
1          1,768.82                   12,839      

In [35]:
import csv

DemographicsData = namedtuple('DemographicsData', ['state', 'county', 'friendly_key', 'countyId', 'pop_without_access', 
                                                   'percent_total_pop', 'pop_density', 'per_capita_income', 
                                                   'urban_pop_without_access', 'percent_urban_pop', 'urban_pop_density', 
                                                   'rural_pop_without_access', 'percent_rural_pop', 'rural_pop_density'])

county_demographics = []

with open('../Data/FCC-2016-Population-Internet.csv') as freader:
    csvreader = csv.DictReader(freader)
    for row in csvreader:
        if 'County' not in row['\ufeffcounty']:
            state = row['\ufeffcounty'].replace(' ', '_').lower()
        else:
            county = row['\ufeffcounty'].replace(' ', '_').lower()
            friendly_key = '%s_%s' % (state, county)
            countyId = friendlyKeyToFips[friendly_key]
            
            
            try:
                pop_without_access = int(row['pop_without_access'].replace(',', ''))
            except ValueError:
                pop_without_access = 0
            
            try:
                percent_total_pop = float(row['percent_total_pop'].strip('%'))
            except ValueError:
                percent_total_pop = 0
            
            try:
                pop_density = float(row['pop_density'].replace(',', ''))
            except ValueError:
                pop_density = 0
            
            try:
                per_capita_income = float(row['per_capita_income'].strip('$').replace(',', ''))
            except ValueError:
                per_capita_income = 0
            
            try:
                urban_pop_without_access = int(row['urban_pop_without_access'].replace(',', ''))
            except ValueError:
                urban_pop_without_access = 0
            
            try:
                percent_urban_pop = float(row['percent_urban_pop'].strip('%').replace(',', ''))
            except ValueError:
                percent_urban_pop = 0
            
            try:
                urban_pop_density = float(row['urban_pop_density'].replace(',', ''))
            except ValueError:
                urban_pop_density = 0
            
            try:
                rural_pop_without_access = float(row['rural_pop_without_access'].replace(',', ''))
            except ValueError:
                rural_pop_without_access = 0
            
            try:
                percent_rural_pop = float(row['percent_rural_pop'].strip('%').replace(',', ''))
            except ValueError:
                percent_rural_pop = 0
            
            try:
                rural_pop_density = float(row['rural_pop_density'].replace(',', ''))
            except ValueError:
                rural_pop_density = 0

            county_demographic = DemographicsData(state, county, friendly_key, countyId,
                                                  pop_without_access, 
                                                  percent_total_pop, 
                                                  pop_density, 
                                                  per_capita_income, 
                                                  urban_pop_without_access, 
                                                  percent_urban_pop, 
                                                  urban_pop_density, 
                                                  rural_pop_without_access, 
                                                  percent_rural_pop, 
                                                  rural_pop_density)
            county_demographics.append(county_demographic)

county_demographics_df = pd.DataFrame(county_demographics, columns=DemographicsData._fields)
print(county_demographics_df.head())
county_demographics_df.to_csv('../Data/CountyDemographics.csv')

     state          county            friendly_key countyId  \
0  alabama  autauga_county  alabama_autauga_county    01001   
1  alabama  baldwin_county  alabama_baldwin_county    01003   
2  alabama  barbour_county  alabama_barbour_county    01005   
3  alabama     bibb_county     alabama_bibb_county    01007   
4  alabama   blount_county   alabama_blount_county    01009   

   pop_without_access  percent_total_pop  pop_density  per_capita_income  \
0               13199               22.0       99.210            24644.0   
1               55717               27.0      128.370            26851.0   
2               13236               51.0       29.563            17350.0   
3               22772               98.0       37.203            18110.0   
4               27824               46.0       93.378            20501.0   

   urban_pop_without_access  percent_urban_pop  urban_pop_density  \
0                       360                1.0            1768.82   
1                     2204

In [34]:
import pandas as pd

us_voter_county_df = pd.DataFrame.from_csv('../Data/US_County_Level_Presidential_Results_08-16.csv', sep=',')
print(us_voter_county_df.head())

                       county  total_2008  dem_2008  gop_2008  oth_2008  \
fips_code                                                                 
26041            Delta County       19064      9974      8763       327   
48295         Lipscomb County        1256       155      1093         8   
1127            Walker County       28652      7420     20722       510   
48389           Reeves County        3077      1606      1445        26   
56017      Hot Springs County        2546       619      1834        93   

           total_2012  dem_2012  gop_2012  oth_2012  total_2016  dem_2016  \
fips_code                                                                   
26041           18043      8330      9533       180       18467      6431   
48295            1168       119      1044         5        1322       135   
1127            28497      6551     21633       313       29243      4486   
48389            2867      1649      1185        33        3184      1659   
56017       

In [7]:
from collections import namedtuple

PopulationCounty = namedtuple('PopulationCounty', ['fips_code', 'population'])

populations = []
url = 'https://www.broadbandmap.gov/broadbandmap/demographic/jun2014/county/ids/%s?format=json'
for fips, county in us_voter_county_df.iterrows():
    response = requests.get(url % fips)
    if 'Response' in response.json():
        populations.append(PopulationCounty(fips_code=fips, population=response.json()['Results'][0]['population']))
    else:
        populations.append(PopulationCounty(fips_code=fips, population=0))

population_df = pd.DataFrame(populations, columns=PopulationCounty._fields)
print(population_df.head())

   fips_code  population
0      26041           0
1      48295           0
2       1127           0
3      48389           0
4      56017           0
